In [ ]:
spark.conf.set("spark.sql.session.timeZone", "UTC")
spark.conf.get("spark.sql.session.timeZone")

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime,timezone

def create_datetime_df():
  date_func = udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), DateType())
  time_func = udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), TimestampType())

  START_DATE = datetime(2021,1,1,0,0,0)
  END_DATE = datetime(2021,7,1,0,0,0)
  df = spark.sql('SELECT sequence({start_date}, {end_date}, 60*15) as timestamp_seq'.format(start_date=int(START_DATE.timestamp()), end_date=int(END_DATE.timestamp())))

  datetime_df = df \
    .withColumn('timestamp',F.explode('timestamp_seq')) \
    .drop('timestamp_seq') \
    .withColumn("id", F.monotonically_increasing_id()) \
    .withColumn('timestamp_string',F.from_unixtime(F.col('timestamp'))) \
    .withColumn('timestamp_datetype',date_func(F.col('timestamp_string'))) \
    .withColumn('timestamp_timestamptype',time_func(F.col('timestamp_string')))
  return datetime_df

In [ ]:
utc_datetime_write = create_datetime_df()

In [ ]:
display(
  utc_datetime_write
)

In [ ]:
utc_datetime_write.write.mode('overwrite').parquet('/dbfs/tmp/datetimetest/')

In [ ]:
spark.conf.set("spark.sql.session.timeZone","Asia/Bangkok")
spark.conf.get("spark.sql.session.timeZone")

In [ ]:
reader = spark.read.parquet('/dbfs/tmp/datetimetest/')

In [ ]:
display(
reader
)

In [ ]:
reader.show()